# Part I. ETL Pipeline for Pre-Processing the Files

#### 1.1) Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### 1.2) Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### 1.3) Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modelling Data with Apache Cassandra 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="images/image_event_datafile_new.jpg">


We are going to create 3 queries to answer the following three questions:

1. Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### 2.1) Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### 2.2) Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """
)
except Exception as e:
    print(e)

#### 2.3) Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### 2.4) Create tables

In [8]:
try:
    rows = session.execute("DROP TABLE table_session_item")
except Exception as e:
    print(e)

try:
    rows = session.execute("DROP TABLE table_user_session")
except Exception as e:
    print(e)
    
try:
    rows = session.execute("DROP TABLE table_song")
except Exception as e:
    print(e)     

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS table_session_item
    (session_id int, item_in_session int, artist_name text, song_title text, song_length float,
    PRIMARY KEY(session_id, item_in_session))
    """)
except Exception as e:
    print(e)
    
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS table_user_session
    (user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text,
    PRIMARY KEY((user_id, session_id), item_in_session))
    """)
except Exception as e:
    print(e) 
    
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS table_song
    (song_title text, user_id int, user_first_name text, user_last_name text,
    PRIMARY KEY(song_title, user_id))
    """)
except Exception as e:
    print(e)

#### 2.5) Insert data

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO table_session_item (session_id, item_in_session, artist_name, song_title, song_length) VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO table_user_session (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO table_song (song_title, user_id, user_first_name, user_last_name) VALUES (%s, %s, %s, %s)"
        
        artist_name, user_first_name, gender, item_in_session, user_last_name, song_length, level, location, session_id, song_title, user_id = line
        item_in_session = int(item_in_session)
        song_length = float(song_length)
        session_id = int(session_id)
        user_id = int(user_id)
        user_first_name = str(user_first_name)
        user_last_name = str(user_last_name)
        
        session.execute(query1, (session_id, item_in_session, artist_name, song_title, song_length))
        session.execute(query2, (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name))
        session.execute(query3, (song_title, user_id, user_first_name, user_last_name))

#### 2.6) Check Table Content

In [10]:
# Testing query 1
print('Testing query 1')
try:
    rows = session.execute("SELECT artist_name, song_title, song_length FROM table_session_item WHERE session_id = 338 AND item_in_session = 4")
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

    
# Testing query 2
print('\nTesting query 2')
try:
    rows = session.execute("SELECT item_in_session, artist_name, song_title, user_first_name, user_last_name FROM table_user_session WHERE user_id = 10 AND session_id = 182")
except Exception as e:
    print(e)

for row in rows:
    print (row.item_in_session, row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

    
# Testing query 3
print('\nTesting query 3')
try:
    rows = session.execute("SELECT user_first_name, user_last_name FROM table_song WHERE song_title = 'All Hands Against His Own'")
except Exception as e:
    print(e)

for row in rows:
    print (row.user_first_name, row.user_last_name)

Testing query 1
Faithless Music Matters (Mark Knight Dub) 495.30731201171875

Testing query 2
0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz

Testing query 3
Jacqueline Lynch
Tegan Levine
Sara Johnson


#### 2.7) Drop the tables before closing out the sessions

In [11]:
## TO-DO: Drop the table before closing out the sessions
try:
    rows = session.execute("DROP TABLE table_session_item")
except Exception as e:
    print(e)

try:
    rows = session.execute("DROP TABLE table_user_session")
except Exception as e:
    print(e)
    
try:
    rows = session.execute("DROP TABLE table_song")
except Exception as e:
    print(e)     

### 2.8) Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()